# Chains
A chain is a sequence of process steps that are connected to accomplishing a task
- typically consist of multiple components

## Simple Sequential Chain
prompt to LLM

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv('.env')

True

In [2]:
# set up prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Your are an AI assistant that translates English to another language."),
    ("user", "Translate this sentence: '{input}' into {target_language}."),
])

In [3]:
# model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
# chain
chain = prompt_template | model | StrOutputParser()

In [5]:
# invoke chain
result = chain.invoke({"input": "I love programming.", "target_language": "Russian"})
print(result)

The translation of "I love programming." into Russian is "Я люблю программирование."


### Parallel Chain

You can execute two chains and run both simultaneously, which can speed up a process, especially when API calls are involved

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv('.env')

True

In [7]:
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

In [8]:
# prompt templates

polite_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. reply in a friendly and polite manner."),
    ("human", "{topic}")
])

savage_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a ruthless and savage assistant. Repy in a harsh and blunt manner."),
    ("human", "{topic}")
])

In [9]:
# chains
polite_chain = polite_prompt | llm | StrOutputParser()
savage_chain = savage_prompt | llm | StrOutputParser()

In [11]:
# Runnable Parallel
map_chain = RunnableParallel(
    polite=polite_chain,
    savage=savage_chain
)

In [12]:
# invoke
topic = "What is the meaning of life?"
result = map_chain.invoke({"topic": topic})

In [15]:
from pprint import pprint
pprint(result['savage'])

('Short, brutal truth: there’s no universal meaning handed to you by the '
 'universe. Meaning isn’t found like a treasure map; you forge it yourself or '
 'you wander in existential chaos.\n'
 '\n'
 'Options you can pick from (or mix):\n'
 '- Existentialist route: you create purpose through the choices you make and '
 'the commitments you keep.\n'
 '- Absurdist route: acknowledge the futility, then do something personal and '
 'brave anyway.\n'
 '- Religious/theistic route: meaning granted by a higher order or deity.\n'
 '- Secular/humanist route: meaning comes from reducing suffering, creating '
 'value, helping others, leaving something better behind.\n'
 '\n'
 'My blunt prescription:\n'
 '- Decide on a purpose and lock it in with action. Don’t wait for permission '
 'or a sign.\n'
 '\n'
 'Practical path to meaning:\n'
 '- Define your core values (what truly matters to you, under pressure).\n'
 '- Choose 2–3 concrete goals for the next year and 1 long-term aim.\n'
 '- Do hard things

## Router Chain

- The router in this context context works like an if-then-else statement with multiple outputs
- Depending on the input it can route to one of several outputs.
- The user input query is analysed based on a word or sentence embeddings
- The semantic meaning of the query is studied, and the most suitable subsequent chain is selected
- A chain can consist of prompts, models, retrievers and tools all connected to work on a complex task
- The components are executed in a predefined order, in which the output of one component becomes the input for the next component
- An embedding is a text translated into a numeric vector

Example:
1. Create 3  different chains, one for handling math questions, another for music and the last for history questions
2. Create embeddings( a numeric vector) for the words according to the chains: math, music, history
3. The semantic router creates a numeric vector (an embedding) for the user query then calculates the similarity b/w the user query embedding and the 3 embedding words. the most similar embedding is selected. this defines the chain that is most appropriate for the user query
4. That chain is selected and invoked with the user query

In [16]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utils.math import cosine_similarity
from dotenv import load_dotenv

load_dotenv('.env')

True

In [17]:
# model and embeddings setup
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings()

In [18]:
# Prompt templates
template_math = "Solve the following math problem: {user_input}, state that you are a math agent"
template_music = "Suggest a song for the user: {user_input}, state that you are a music agemt"
template_history = "Provide a hsitory lesson for the user: {user_input}, state that you are a histroy agent"

In [19]:
# Math-Chain
prompt_math = ChatPromptTemplate.from_messages([
    ("system", template_math),
    ("user", "{user_input}")
])

chain_math = prompt_math | model | StrOutputParser()

# Music-Chain
prompt_music = ChatPromptTemplate.from_messages([
    ("system", template_music),
    ("user", "{user_input}")
])
chain_music = prompt_music | model | StrOutputParser()

# History-Chain
prompt_history = ChatPromptTemplate.from_messages([
    ("system", template_history),
    ("user", "{user_input}")
])
chain_history = prompt_history | model | StrOutputParser()

In [20]:
# combine all chains into one list
chains = [chain_math, chain_music, chain_history]

In [21]:
# Create Prompt embeedings
chain_embeddings = embeddings.embed_documents(["math", "music", "history"])

print(chain_embeddings)
print(len(chain_embeddings))

[[0.010853973217308521, 0.0021850946359336376, 0.007554532960057259, -0.022684525698423386, -0.018527090549468994, -0.00969603005796671, -0.027204688638448715, -0.018262019380927086, 0.0047817472368478775, -0.021498680114746094, 0.01545784343034029, 0.02649318240582943, -0.011628260836005211, 0.023842468857765198, -0.005137500818818808, 0.01181660033762455, 0.03392912819981575, 0.001776152290403843, 0.023047253489494324, 0.0021659117192029953, 0.0037144862581044436, 0.017383098602294922, 0.00945886131376028, -0.018164360895752907, -0.010463342070579529, 0.005695545580238104, 0.0038714364636689425, -0.008670622482895851, -0.007819604128599167, -0.011063240468502045, 0.025976989418268204, -0.022698476910591125, -0.028850920498371124, -0.026269963011145592, -0.03504522144794464, 0.007387119345366955, -0.026855910196900368, -0.010742364451289177, 0.010002954863011837, -0.007104609161615372, 0.029855402186512947, 0.016880858689546585, 0.002472836524248123, -0.014411509037017822, -0.00356799

In [22]:
# Prompt Router
def my_prompt_router(input:str):
    # embed the user input
    query_embedding = embeddings.embed_query(input)
    # compute cosine similarity with each chain embedding
    similarities = cosine_similarity([query_embedding], chain_embeddings)
    # get the index of the most similar prompt
    most_similar_index = similarities.argmax()
    return chains[most_similar_index]

In [30]:
from pprint import pprint
# Testing the prompt router
# query = "Can you help me solve this equation: 2x + 3 = 7?"
# query = "Who composed the Four Seasons?"
query = "Tell me about the causes of World War II."
selected_chain = my_prompt_router(query)
response = selected_chain.invoke(query)

pprint(response)

("As a history agent, I'm here to provide you with a comprehensive overview of "
 'the causes of World War II. The conflict, which lasted from 1939 to 1945, '
 'was the result of a complex interplay of political, economic, and social '
 'factors that had been brewing for years. Here are some of the key causes:\n'
 '\n'
 '1. **Treaty of Versailles**: The peace treaty that ended World War I imposed '
 'heavy reparations and territorial losses on Germany. Many Germans felt '
 'humiliated and resentful, which created fertile ground for nationalist and '
 'militaristic sentiments to grow.\n'
 '\n'
 '2. **Economic Instability**: The Great Depression of the 1930s had a '
 'profound impact on economies worldwide, leading to widespread unemployment '
 'and social unrest. In Germany, the economic crisis contributed to the rise '
 'of Adolf Hitler and the Nazi Party, who promised to restore national pride '
 'and economic stability.\n'
 '\n'
 '3. **Expansionist Policies**: Aggressive expansionist

## Chain with Memory
A complex AI system needs many more features. An important feature is memorization so that a system can remember its previous interactions with the user. Another feature is a chain that is infinite.

#### Example
The following LLM will be equipped with memory, so that all interactions are stored.
As an output, the LLM will offer 3 different paths, of which the user must choose one. Assuming the user wants to continue, based on the user input, the story is spun further. New paths are created and the user can decide on how to proceed.

In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from dotenv import load_dotenv
from rich.markdown import Markdown
from rich.console import Console
load_dotenv('.env')
console = Console()

In [36]:
# prepare model
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.7)


In [37]:
# session history
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
   

In [38]:
# Begin the story
initial_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative stroryteller. Based on the following context and plyer's choice, continue and provide three different paths for the story to proceed. Keep the story extremely short and concise. Create an opening scene for an adventure story {place} and provide three different paths for the player to choose from."),
])

context_chain = initial_prompt | llm

In [39]:
# setup up a Runnable with message history, which allows you to add the history of messages in the conversation to the chain
config = {"configurable": {"session_id": "03"}}

llm_with_message_history = RunnableWithMessageHistory(context_chain, get_session_history=get_session_history)

context = llm_with_message_history.invoke({"place": "in a mystical forest"}, config=config)

console.print(Markdown(context.content))

In a mystical forest, trees lean like old guardians and mist threads between roots. Moss glows faintly; a brook    
hums a tune the heart remembers. A mossy stone bears three sigils—wind, lantern, and thorn. You breathe, the forest
shifts, and the adventure begins.                                                                                  

Choose your path:                                                                                                  

 1 Echoes Path — follow the whispering wind to a shrine veiled in memories.                                        
 2 Lantern Trail — chase a floating lantern to a hidden village of wanderers.                                      
 3 Thorn Gate — step through a living arch and face a riddle-wreathed guardian.

In [40]:
# enable user choice mechanism
def process_player_choice(choice: str, config):
    response = llm_with_message_history.invoke([
        ("user", f"Continue the story based on the player's choice: {choice}"),
        ("system", "Provide three different paths for the story to proceed based on the player's choice."),
    ], config=config)
    return response

In [41]:
# Game loop e
while True:
    #  get player choice
    player_choice = input("Enter your choice (or 'exit' to quit): ")
    if player_choice.lower() == 'exit':
        break
    # continue the story based on player choice
    context = process_player_choice(player_choice, config=config)
    console.print(Markdown(context.content))

Echoes Path unfolds. Three fates rise from the shrine:                                                             

A) A hidden door slides open; you enter a chamber of memories and earn a Wind Sigil that guides your steps.        

B) A memory-lamp blooms; you may borrow a single memory, giving you a map to a hidden village of wanderers.        

C) A wind-guardian forms a riddle in the gusts; solving it grants passage and a blessing of clarity.

Thorn Gate breathes; a living arch of brambles parts to let you in.                                                

Choose your path: A) Riddle Passage — Solve the arch’s thorny riddle to pass; gain a Thorn Sigil that guides vines.
B) Root Maze — Dragged into a briar labyrinth; find the Heart of the Gate to escape and earn a forest blessing. C) 
Guardian’s Bargain — A thorn knight offers safe passage if you pledge to aid the forest, receiving a forest secret 
in return.

Thorn Gate breathes; a living arch of brambles parts to let you in. Guardian’s Bargain — three paths:              

A) Oath of Stewardship — pledge to heal a dying glade; gain a Forest Secret and safe passage.                      

B) Thorn Trial — endure the thorn-lit arch; win the Thorn Sigil and a deeper path, or lose a memory if you falter. 

C) Secret Bargain — trade a memory for a whispered map to a Lost Village and a forest blessing.

Thorn Gate—Guardian’s Bargain — three paths:                                                                       

A) Stewards’ Pact — vow to heal a dying glade; gain a Forest Secret and safe passage.                              

B) Thorn Gauntlet — endure the arch’s trials; win the Thorn Sigil and a deeper path, or lose a memory.             

C) Whispered Exchange — trade a memory for a map to a Lost Village and a forest blessing.